In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from bioio import BioImage
import bioio_ome_zarr
import matplotlib.pyplot as plt
from skimage import exposure
from skimage.transform import resize

In [ ]:
base_path = "/mnt/gdrive/ThNe/development_retina/"

In [ ]:
fused_path = base_path + "fused_images_n5/"

In [ ]:
dapi_path = 'dapi'
mg_path = 'microglia'
blood_vessels = 'blood'

In [ ]:
p15 = "P15/F/L/retina_Age_P15_Sex_F_Side_L_Animal_1.zarr"

In [ ]:
img_path = fused_path + p15
img = BioImage(img_path, reader=bioio_ome_zarr.Reader)

In [ ]:
import zarr

# Open the Zarr file (can be a directory or .zarr file)
z = zarr.open(img_path, mode='r')

# List the groups and arrays inside
z.tree()

In [ ]:
import zarr
import tifffile
import os
import gc  # for freeing memory

# Setup
base_path = "/mnt/gdrive/ThNe/development_retina/"
dapi_path = "dapi"
mg_path = "microglia"
blood_vessels = "blood"
zarr_file = "P15/F/L/retina_Age_P15_Sex_F_Side_L_Animal_1.zarr"
zarr_path = os.path.join(base_path, "fused_images_n5",zarr_file)

# Pixel size in microns
pix_size = {
    'x': 2*0.339024328759285,
    'y': 2*0.339024328759285,
    'z': 2*0.9
}

# Map channel index to subfolder
channel_map = {
    0: dapi_path,
    1: mg_path,
    2: blood_vessels
}

# Open the Zarr
z = zarr.open(zarr_path, mode='r')
arr = z['0']  # shape: (1, 3, Z, Y, X)

# Process each channel one by one
for ch_idx, subfolder in channel_map.items():
    print(f"Processing channel {ch_idx} → {subfolder}")
    
    # Load data
    data = arr[0, ch_idx, :, :, :]  # shape: (Z, Y, X)
    
    # Get dimensions
    z_max_pix, y_max_pix, x_max_pix = data.shape

    # Build output path
    tif_filename = f"{subfolder}_Age_P15_Sex_F_Side_L_Animal_1.tif"
    out_path = os.path.join(base_path, subfolder, "P15", "F", "L")
    os.makedirs(out_path, exist_ok=True)
    full_tif_path = os.path.join(out_path, tif_filename)

    # Metadata with physical size info
    metadata = {
        # Physical dimensions in microns
        "ExtendMinX": 0,
        "ExtendMaxX": x_max_pix * pix_size['x'],
        "ExtendMinY": 0,
        "ExtendMaxY": y_max_pix * pix_size['y'],
        "ExtendMinZ": 0,
        "ExtendMaxZ": z_max_pix * pix_size['z'],
        # Dimensions in pixels
        "SizeX": x_max_pix,
        "SizeY": y_max_pix,
        "SizeZ": z_max_pix,
    }

    # Save as TIFF
    tifffile.imwrite(
        full_tif_path,
        data,
        metadata=metadata,
        dtype=data.dtype
    )
    print(f"Saved {full_tif_path}")

    # Free memory
    del data
    gc.collect()
